In [21]:
workspace()
using Bolas
using Derivatives
using BolaIntervalo


    +(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:216
is ambiguous with: 
    +(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:221.
To fix, define 
    +(Bola,Bola)
before the new definition.
    +(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:216
is ambiguous with: 
    +(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:221.
To fix, define 
    +(Bola,Bola)
before the new definition.
    +(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:221
is ambiguous with: 
    +(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:216.
To fix, define 
    +(Bola,Bola)
before the new definition.
    +(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:221
is ambiguous with: 
    +(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:216.
To fix, define 
    +(Bola,Bola)
before the new definition.
    -(Bola,Any) at /home/pablo1/Documents/Ta

In [14]:
A=Bola(4,1)
B=Bola(2,1)
N=Bola(0.9999942779541016,9.5367431640625e-6)

function wilkinson20(x)
    
    w=1
    
    for i=1:20
        w*=(x-i)
    end
    
    return(w)
end
        

wilkinson20 (generic function with 1 method)

In [57]:
function revisar_cercanas(A,B,tol) #revisa si una raíz dada está cerca de las demás
    
    for i = 1:length(B)
        
        if distancia(A,B[i]) <= tol
            return(true) 
        end
        
        
    end
    
    return(false)
end

function quitar_repetidas(A,tol) #remueve raices que esten mas cerca que una cierta tolerancia
   
    temp=Bola[]
    
    for i=1:length(A)
        
        if !(revisar_cercanas(A[i],temp,tol))
            push!(temp,A[i])
        end
        
    end
    
    return(temp)
end

function quitar_no_deseadas(F::Function,B,x) #Funcion para quitar bolas donde la derivada de la funcion se hace cero
    
    temp=Bola[]
    dF(x)=F(makex(x)).d
    
    for i=1:length(B)
        if !contiene(dF(B[i]),x)
            push!(temp,B[i])
        end
    end
    
    return(temp)
end

function quitar_no_utiles(F::Function,B,x) #Se encarga de quitar bolas que no contienen ceros de la funcion
    
    temp=Bola[]
    
    for i=1:length(B)
        if contiene(F(B[i]),x)
            push!(temp,B[i])
        end
    end
    
    return(temp)
end



function operador_de_newton_bola(F::Function,B::Bola) 
    
    dF(x)=F(makex(x)).d
    m=Bola(B.centro,1e-15) #Le asigno un radio mínimo al punto m 
    
    
    return (m-F(m)/(dF(B)))
    
end   

function metodo_newton_bola(F::Function, B::Bola)
    
    X=[B]
    
    i=0
    tolbis=1e-5
    tolradio=1e-10
    tolsimilar = 1e-2
    
    while(X[1].radio>tolbis)
        
       X=bisectar_bolas(X)
       X=quitar_no_utiles(F,X,0)
    
        
        i+=1
        
        if i==50
            break
        end
        
        X
 
    end
    
    @show X=quitar_no_deseadas(F,X,0)
    
    j=0
    
    radio_biseccion = X[1].radio
    radio_promedio=1
    
    while(tolradio<radio_promedio)
        
        radio_promedio=0
        
        
        for i=1:length(X)
            
            temp=operador_de_newton_bola(F,X[i])
            
            if interseccion_de_bolas(X[i],temp)==nothing
                X[i]=interseccion_de_bolas(B,temp)
            else
                X[i]=interseccion_de_bolas(X[i],temp)
            end
            
            radio_promedio+=(X[i].radio)
        end
         
        radio_promedio/=length(X)
        
        j+=1
        
        if j==20
            break
        end
    end
    
    

    ans = X
    
    for i=1:length(ans) 
        println("Posible raíz en $(ans[i].centro) ± $(ans[i].radio), $(F(ans[i]))")
    end
    
end


f(x)=(x-1)*(x-2)*(x-3)*(x-4)*(x-5)
g(x)=(x-4)*(exp(x)-1)*(x-3)
M=metodo_newton_bola(f,Bola(6,10))

X = quitar_no_deseadas(F,X,0) => [Bola(0.9999904632568359,9.5367431640625e-6),Bola(1.000009536743164,9.5367431640625e-6),Bola(1.9999942779541016,9.5367431640625e-6),Bola(2.999998092651367,9.5367431640625e-6),Bola(4.000001907348633,9.5367431640625e-6),Bola(5.000005722045898,9.5367431640625e-6)]
Posible raíz en 1.000044141909915 ± 2.710823653965378e-10, Bola(0.0010593084155618414,6.505976758378499e-9)
Posible raíz en 0.9999558387316126 ± 2.7105997426428735e-10, Bola(-0.001059967955192178,6.506636480424848e-9)
Posible raíz en 1.999999910590431 ± 2.165795851569183e-15, Bola(5.364574544609504e-7,1.2994779369558614e-14)
Posible raíz en 2.999999984741211 ± 1.8007593497524146e-15, Bola(-6.103515559630067e-8,7.203037728738622e-15)
Posible raíz en 4.0 ± 2.000000000000451e-15, Bola(-0.0,1.2000000000002846e-14)
Posible raíz en 5.000000000366319 ± 3.739307731091528e-15, Bola(8.791666068534296e-9,8.974338568317621e-14)


In [62]:
operador_de_newton_bola(f,Bola(5.000000228947109,1.3621190883846948e-7))


Bola(5.000000009157989,7.784062269786418e-11)

In [67]:
contiene(f(Bola(5.000000228947109,1.3621190883846948e-7)),0)

false

In [31]:
operador_de_newton_bola(f,Bola(BigFloat(17.000045776367188),BigFloat(7.62939453125e-5)))

Bola(1.700000000325142377049077502723301958906439662258370670382541753535895760327455e+01 with 256 bits of precision,5.744931712241313052974049658227678358362202936489743807669440160575048763987605e+20 with 256 bits of precision)

In [8]:
function NewtonMethod(g::Function,x,tol) #x en este caso es la "adivinanza" inicial
    
    i = 0
    
    while(abs(g(x))>tol)
        

        x-=GetFunctionValue(g,x)/GetDerivativeValue(g,x)
        
        i+=1
        if(i==70)
            break
        end
    end
    
    #return("Posible raíz en $x, la función vale f(x)=$(g(x))  ")
    return(x)
end

NewtonMethod(f,1+1e-15,1e-15)

1.0281872039930624

LoadError: M not defined
while loading In[2], in expression starting on line 30

In [62]:
M=[Bola(3),Bola(2),Bola(3),Bola(4),Bola(2),Bola(3.1)]

6-element Array{Bola,1}:
 Bola(3,1)  
 Bola(2,1)  
 Bola(3,1)  
 Bola(4,1)  
 Bola(2,1)  
 Bola(3.1,1)

In [50]:
function quitar_bolas_similares(B,tol::Real)
    
    return(temp)
end

quitar_bolas_similares (generic function with 1 method)

false

In [54]:
sort(M)

LoadError: `isless` has no method matching isless(::Bola, ::Bola)
while loading In[54], in expression starting on line 1

In [47]:
57847/1.16

49868.10344827586

In [48]:
150000/15

10000.0

In [106]:
M=Bola([1,1],1)
N=Bola([1,2],1)

Bola([1,2],1)

In [110]:
M-N

Bola([0,-1],2)

In [9]:
A=Bola(2,1)
B=Bola(3,2)

A/B

Bola(0.6666666666666666,6.333333333333334)

In [10]:
using Intervals

In [11]:
Interval(1,3)/Interval(1,5)

Interval(0.19999999999999998,3.0)